In [10]:
import os
from dotenv import load_dotenv
import requests
load_dotenv()


True

In [16]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine,text as sql_text


db_username = 'myuser'
db_password = 'root'
db_host = 'localhost'
db_port = '5432'
db_name = 'medwell_db'

connection_string = f'postgresql+psycopg2://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}'
engine = create_engine(connection_string)

In [18]:
con=engine.connect()

In [12]:
import pandas as pd

In [19]:
df=pd.read_sql_query(sql=sql_text('''
    select date_of_report,report_type,report_data ,summary from patient_report r join patient_reportdetail d on r.id=d.report_id 
    order by r.submitted_at desc
    limit 5;
    '''),con= conn)

In [20]:
df

,date_of_report,report_type,report_data,summary
0,3/8/24,blood_report,"{'pcv': {'max': 50, 'min': 36, 'unit': '%', 'v...",The report shows abnormal values in RBC morpho...
1,3/8/24,blood_report,"{'pcv': {'max': 50, 'min': 36, 'unit': '%', 'v...","Abnormal values included hemoglobin (12.1 g%),..."
2,13/8/24,blood_report,"{'pcv': {'max': 50, 'min': 36, 'unit': '%', 'v...","Abnormal values include Hemoglobin, RBC Count,..."
3,11/09/22,blood_report,"{'pcv': {'max': 50, 'min': 36, 'unit': '%', 'v...","Hemoglobin, RBC Count, PCV, Neutrophils, and L..."


In [21]:
def generate_report_text(report_data):
    report_text = ""
    for test, details in report_data.items():
        min_val = details['min']
        max_val = details['max']
        unit = details['unit']
        value = details['value']
        if value==-1:continue
        
        value_text = f"{value} {unit}" if value != -1 else "Not available"
        
        report_text += f"{test.capitalize()}: Normal range {min_val}-{max_val} {unit}, Current value: {value_text}\n"
    
    return report_text

In [22]:
df["report_data"]=df.report_data.apply(generate_report_text)

In [23]:
df

,date_of_report,report_type,report_data,summary
0,3/8/24,blood_report,"Pcv: Normal range 36-50 %, Current value: 38 %...",The report shows abnormal values in RBC morpho...
1,3/8/24,blood_report,"Pcv: Normal range 36-50 %, Current value: 38 %...","Abnormal values included hemoglobin (12.1 g%),..."
2,13/8/24,blood_report,"Pcv: Normal range 36-50 %, Current value: 31 %...","Abnormal values include Hemoglobin, RBC Count,..."
3,11/09/22,blood_report,"Pcv: Normal range 36-50 %, Current value: 32 %...","Hemoglobin, RBC Count, PCV, Neutrophils, and L..."


In [24]:
from langchain_groq import ChatGroq

In [25]:
llm=ChatGroq(api_key=os.environ['GROQ'],model="llama3-70b-8192")

In [26]:
llm.invoke("who is doctor strange")

AIMessage(content='Doctor Strange is a fictional superhero appearing in American comic books published by Marvel Comics. He is a sorcerer who uses magical powers to protect the Earth and other dimensions from magical threats.\n\nDoctor Strange\'s real name is Stephen Vincent Strange. He was once a brilliant and arrogant neurosurgeon who lost the use of his hands in a car accident. Desperate to regain the use of his hands, Strange searched for a cure and eventually found himself in the Himalayas, where he encountered the Ancient One, a powerful sorcerer who became his mentor.\n\nThe Ancient One taught Strange the mystic arts, and he quickly became a powerful sorcerer in his own right. Using his newfound abilities, Strange created a magical sanctum in New York City and set out to defend the Earth against magical threats.\n\nDoctor Strange\'s powers come from his ability to cast spells, summon magical entities, and wield magical artifacts. He is also a skilled martial artist and strategis

In [27]:
from pandasai import SmartDataframe,

In [29]:
agent = SmartDataframe(df,config = {'llm':llm})

In [33]:
import warnings

In [34]:
warnings.filterwarnings("ignore")

In [45]:
resp=agent.chat('what is my average rbc count.Give a single value',output_type="string")

--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\chouh\AppData\Local\Programs\Python\Python311\Lib\logging\__init__.py", line 1113, in emit
    stream.write(msg + self.terminator)
  File "C:\Users\chouh\AppData\Local\Programs\Python\Python311\Lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeEncodeError: 'charmap' codec can't encode character '\u03bc' in position 230: character maps to <undefined>
Call stack:
  File "C:\Users\chouh\AppData\Local\Programs\Python\Python311\Lib\runpy.py", line 198, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\chouh\AppData\Local\Programs\Python\Python311\Lib\runpy.py", line 88, in _run_code
    exec(code, run_globals)
  File "B:\Major Project\major\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "B:\Ma

In [46]:
resp

'Unfortunately, I was not able to answer your question, because of the following error:\n\ndivision by zero\n'